In [0]:
from datetime import date, timedelta

# Remove this before running Data Factory Pipeline
start_date = date.today() - timedelta(1)

bronze_adls = "abfss://bronze@dataluke124.dfs.core.windows.net/"
silver_adls = "abfss://silver@dataluke124.dfs.core.windows.net/"

In [0]:
from pyspark.sql.functions import col, isnull, when
from pyspark.sql.types import TimestampType
from datetime import date, timedelta

In [0]:
# Load the JSON data into a Spark DataFrame
df = spark.read.option("multiline", "true").json(f"{bronze_adls}{start_date}_earthquake_data.json")

In [0]:
df

DataFrame[geometry: struct<coordinates:array<double>,type:string>, id: string, properties: struct<alert:string,cdi:double,code:string,detail:string,dmin:double,felt:bigint,gap:double,ids:string,mag:double,magType:string,mmi:double,net:string,nst:bigint,place:string,rms:double,sig:bigint,sources:string,status:string,time:bigint,title:string,tsunami:bigint,type:string,types:string,tz:string,updated:bigint,url:string>, type: string]

In [0]:
df.head()

Row(geometry=Row(coordinates=[-121.9165, 38.1118333333333, 25.87], type='Point'), id='nc75186056', properties=Row(alert=None, cdi=None, code='75186056', detail='https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=nc75186056&format=geojson', dmin=0.03164, felt=None, gap=97.0, ids=',nc75186056,', mag=1.73, magType='md', mmi=None, net='nc', nst=16, place='10 km  of Pittsburg, CA', rms=0.13, sig=46, sources=',nc,', status='reviewed', time=1748130484280, title='M 1.7 - 10 km  of Pittsburg, CA', tsunami=0, type='earthquake', types=',nearby-cities,origin,phase-data,scitech-link,', tz=None, updated=1748166738755, url='https://earthquake.usgs.gov/earthquakes/eventpage/nc75186056'), type='Feature')

In [0]:

# Reshape earthquake data
df = (
    df
    .select(
        'id',
        col('geometry.coordinates').getItem(0).alias('longitude'),
        col('geometry.coordinates').getItem(1).alias('latitude'),
        col('geometry.coordinates').getItem(2).alias('elevation'),
        col('properties.title').alias('title'),
        col('properties.place').alias('place_description'),
        col('properties.sig').alias('sig'),
        col('properties.mag').alias('mag'),
        col('properties.magType').alias('magType'),
        col('properties.time').alias('time'),
        col('properties.updated').alias('updated')
    )
)

In [0]:
df

DataFrame[id: string, longitude: double, latitude: double, elevation: double, title: string, place_description: string, sig: bigint, mag: double, magType: string, time: bigint, updated: bigint]

In [0]:
df.head()

Row(id='nc75186056', longitude=-121.9165, latitude=38.1118333333333, elevation=25.87, title='M 1.7 - 10 km  of Pittsburg, CA', place_description='10 km  of Pittsburg, CA', sig=46, mag=1.73, magType='md', time=1748130484280, updated=1748166738755)

In [0]:
# Validate data: Check for missing or null values
df = (
    df
    .withColumn('longitude', when(isnull(col('longitude')), 0).otherwise(col('longitude')))
    .withColumn('latitude', when(isnull(col('latitude')), 0).otherwise(col('latitude')))
    .withColumn('time', when(isnull(col('time')), 0).otherwise(col('time')))
)

In [0]:

# Convert 'time' and 'updated' to timestamp from Unix time
df = (
    df
    .withColumn('time', (col('time') / 1000).cast(TimestampType()))
    .withColumn('updated', (col('updated') / 1000).cast(TimestampType()))
)
   

In [0]:
df.head()

Row(id='nc75186056', longitude=-121.9165, latitude=38.1118333333333, elevation=25.87, title='M 1.7 - 10 km  of Pittsburg, CA', place_description='10 km  of Pittsburg, CA', sig=46, mag=1.73, magType='md', time=datetime.datetime(2025, 5, 24, 23, 48, 4, 280000), updated=datetime.datetime(2025, 5, 25, 9, 52, 18, 755000))

In [0]:
# Save the transformed DataFrame to the Silver container
silver_output_path = f"{silver_adls}earthquake_events_silver/"

In [0]:
# Append DataFrame to Silver container in Parquet format
df.write.mode('append').parquet(silver_output_path)
     